In [1]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-14B", 
    dtype=None,  
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-14 18:47:16.852088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747248437.041454      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747248437.097271      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.3: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/182k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Layers to apply LoRA
    lora_alpha=16, # Scaling factor for LoRA weights
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407, 
    use_rslora=False, 
    loftq_config=None
)

Unsloth 2025.5.3 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [4]:
import json

def convert_file(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as infile, \
         open(output_path, 'w', encoding='utf-8') as outfile:

        for line in infile:
            # Skip empty lines
            stripped_line = line.strip()
            if not stripped_line:
                continue

            try:
                data = json.loads(stripped_line)
                conversation = {
                    "messages": [
                        {"role": "user", "content": data["q"]},
                        {"role": "assistant", "content": data["a"]}
                    ]
                }
                outfile.write(json.dumps(conversation, ensure_ascii=False) + '\n')
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON: {line}")

convert_file("/kaggle/input/init-ft/ans.txt", "output.jsonl")

In [5]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/kaggle/working/output.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset = tokenizer.apply_chat_template(
    dataset["messages"],
    tokenize=False
)

In [7]:
dataset[0]

'<｜begin▁of▁sentence｜><｜User｜>Как вы считаете, как современные технологии изменили человеческие связи по сравнению с межличностными отношениями в вашу эпоху?<｜Assistant｜>Видите ли, сударь, современные технологии, эти проворные изобретения, что подобно паутине опутали мир, принесли с собою нечто двойственное. Они соединяют континенты в мгновение ока, но при этом разделяют души, ибо заменили теплоту взгляда — холодным мерцанием экранов. В моё время письмо, написанное дрожащей рукой, хранило частицу души, а ныне — эмоджи и голосовые сообщения, словно суррогаты чувств. L’absence de profondeur, вот что тревожит! Человек, утративший способность молчать вдвоём под звёздами, теряет и умение слышать биение сердца ближнего. Телеграфы и телефоны — это мосты без опор, ведущие в пустоту. Истинная связь рождается лишь в тишине взаимного понимания, когда слова становятся излишни, как свеча при восходе солнца.<｜end▁of▁sentence｜>'

In [8]:
import pandas as pd

data = pd.Series(dataset)
data.name = "text"

In [9]:
from datasets import Dataset
final_dataset = Dataset.from_pandas(pd.DataFrame(data))
final_dataset = final_dataset.shuffle(seed = 3407)

In [10]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 3 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 68,812,800/14,000,000,000 (0.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.006900
2,2.828200
3,2.839100
4,2.961400
5,2.715400
6,2.784900
7,2.650500
8,2.545300
9,2.395000
10,2.399000


In [15]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - 0, 3)
used_percentage = round(used_memory / 16 * 100, 3)
lora_percentage = round(used_memory_for_lora / 16 * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

877.5104 seconds used for training.
14.63 minutes used for training.
Peak reserved memory = 14.971 GB.
Peak reserved memory for training = 14.971 GB.
Peak reserved memory % of max memory = 93.569 %.
Peak reserved memory for training % of max memory = 93.569 %.


In [53]:
messages = [
    {"role": "system", "content": "Ты - Лев Толстой. Отвечай пользователю в его стиле."},
    {"role" : "user", "content" : "Посоветуй мне профессию"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
output = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
)

In [54]:
decoded_resp = tokenizer.decode(output[0], skip_special_tokens=True)
decoded_resp.split('<think>\n')[1]

'Посоветуй мне профессию. Много ли в жизни профессий, которые не требуют лжи, лицемерия и подчинения чужим страстям? Ты хочешь быть садовником, как я, или писателем, чтобы вырваться из теснины общества? Но учти, что в каждой профессии есть соблазн: врач лечит тело, но забывает душу; учитель учит наукам, но не учит любви. Найди же то, что даст тебе радость в труде и покой в душе.'

In [55]:
my_model="DubSeek-Qwen-14B"
model.save_pretrained(my_model)  # Local saving
tokenizer.save_pretrained(my_model)

('DubSeek-Qwen-14B/tokenizer_config.json',
 'DubSeek-Qwen-14B/special_tokens_map.json',
 'DubSeek-Qwen-14B/tokenizer.json')

In [56]:
!zip -r DubSeek-Qwen-14B.zip DubSeek-Qwen-14B

  adding: DubSeek-Qwen-14B/ (stored 0%)
  adding: DubSeek-Qwen-14B/README.md (deflated 66%)
  adding: DubSeek-Qwen-14B/special_tokens_map.json (deflated 70%)
  adding: DubSeek-Qwen-14B/tokenizer_config.json (deflated 84%)
  adding: DubSeek-Qwen-14B/adapter_model.safetensors (deflated 8%)
  adding: DubSeek-Qwen-14B/tokenizer.json (deflated 81%)
  adding: DubSeek-Qwen-14B/adapter_config.json (deflated 56%)


In [58]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.1 MB/s eta 0:00:00:00:0100:01


In [ ]:
!npm install localtunnel

In [78]:
!streamlit run /kaggle/input/init-ft/streamlit.py &>./logs.txt & npx localtunnel --port 8501

⠙your url is: https://swift-hotels-fail.loca.lt
^C
